*This code sets up a web application using Streamlit and Ngrok to perform sentiment analysis on user-provided text and CSV files. The application provides a user-friendly interface to analyze individual text sentiments and clean text data. It also enables users to upload CSV files, perform sentiment analysis on text data within the files, and download the results.*

# 1.Setting up Ngrok Authtoken:

**from pyngrok import ngrok:** Imports the Ngrok module from the Pyngrok library, which is used to
create secure tunnels to your localhost.

**ngrok.set_auth_token(...):** Sets your Ngrok authentication token, allowing you to connect to Ngrok's service securely. This token is specific to your Ngrok account.

In [3]:
!pip install pyngrok
from pyngrok import ngrok

# Set your Ngrok authtoken
ngrok.set_auth_token('2lnbNYEGVlOilStHi6lfAETdtpP_kMCx4REcfZ9haktvp9mF')


# 2.Starting the streamlit app

**!streamlit run app.py &>/dev/null&:**  This command starts the Streamlit application defined in the file app.py. The &>/dev/null& part redirects the output to /dev/null, essentially hiding the output from the terminal.

In [4]:
# Start the Streamlit app
!streamlit run app.py &>/dev/null&

# 3.Exposing the Streamlit app using Ngrok:

**public_url = ngrok.connect(addr='8501'):**  This line uses Ngrok to create a public URL that maps to your local Streamlit app running on port 8501. addr='8501' specifies the address to be tunneled.

**print(f'Access the app here: {public_url}'): ** Prints the public URL, allowing you to access your Streamlit app from any browser.

In [5]:
# Expose the Streamlit app using Ngrok
# Use addr instead of port for HTTP tunnels
public_url = ngrok.connect(addr='8501')
print(f'Access the app here: {public_url}')


Access the app here: NgrokTunnel: "https://dab1-34-122-214-116.ngrok-free.app" -> "http://localhost:8501"


# 4.Installing Necessary Libraries:

**!pip install streamlit:** Installs Streamlit, the Python library used to create interactive web apps.

**!pip install pyngrok:**  Installs Pyngrok, the Python wrapper for Ngrok, used to create secure tunnels.

**!pip install textblob clean-text pandas:** Installs additional libraries:

**TextBlob:** For natural language processing tasks, such as sentiment analysis.

**clean-text:** For cleaning text data (removing punctuation, stopwords, etc.).

**Pandas: **For data manipulation and analysis, often used with CSV files.

In [6]:
# Step 1: Install necessary libraries
!pip install streamlit
!pip install pyngrok
!pip install textblob clean-text pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=aeec7b84f231967f29a4aa495f53897d4a439e6e47d9e481cc9a1adfd05107a7
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb

# 5.Creating the Streamlit App (app.py):

**with open('app.py', 'w') as f:** Opens a file named app.py in write mode ('w'), creating it if it doesn't exist.

**f.write("""..."""):** rites the Streamlit app code (enclosed within the triple quotes) into the app.py file.

In [7]:
Save the Streamlit app code as a Python file
with open('app.py', 'w') as f:
    f.write("""
from textblob import TextBlob
import pandas as pd
import streamlit as st
import cleantext

st.header('Sentiment Analysis')

# Individual Text Analysis
with st.expander('Analyze Text'):
    text = st.text_input('Enter text for sentiment analysis:')
    if text:
        blob = TextBlob(text)
        st.write('Polarity:', round(blob.sentiment.polarity, 2))
        st.write('Subjectivity:', round(blob.sentiment.subjectivity, 2))

    pre = st.text_input('Clean your text:')
    if pre:
        cleaned_text = cleantext.clean(
            pre, clean_all=False, extra_spaces=True, stopwords=True, lowercase=True, numbers=True, punct=True
        )
        st.write('Cleaned Text:', cleaned_text)

# CSV Analysis
with st.expander('Analyze CSV'):
    upl = st.file_uploader('Upload your CSV file:', type=['csv', 'xlsx'])

    # Function to calculate sentiment polarity
    def score(x):
        return TextBlob(x).sentiment.polarity

    # Function to assign sentiment label
    def analyze(x):
        if x >= 0.5:
            return 'Positive'
        elif x <= -0.5:
            return 'Negative'
        else:
            return 'Neutral'

    if upl:
        try:
            if upl.name.endswith('.csv'):
                df = pd.read_csv(upl)
            else:
                df = pd.read_excel(upl)

            if 'tweets' not in df.columns:
                st.error("The uploaded file must contain a 'tweets' column.")
            else:
                if 'Unnamed: 0' in df.columns:
                    del df['Unnamed: 0']

                df['score'] = df['tweets'].apply(score)
                df['analysis'] = df['score'].apply(analyze)
                st.write(df.head(10))

                # Convert the dataframe to CSV for download
                @st.cache_data
                def convert_df(df):
                    return df.to_csv(index=False).encode('utf-8')

                csv = convert_df(df)

                st.download_button(
                    label="Download data as CSV",
                    data=csv,
                    file_name='sentiment.csv',
                    mime='text/csv',
                )
        except Exception as e:
            st.error(f"Error processing file: {e}")
   """)

# 6.Running the Streamlit App:

This section repeats the steps of setting up Ngrok, running the Streamlit app, and exposing it, similar to the first part of the script.

In [8]:
# Step 3: Run the Streamlit app with Ngrok
from pyngrok import ngrok

# Set your Ngrok authtoken (replace with your actual token)
ngrok.set_auth_token('YOUR_NGROK_AUTH_TOKEN')

# Start the Streamlit app
!streamlit run app.py &>/dev/null&

# Expose the Streamlit app using Ngrok
# Use addr instead of port for HTTP tunnels
public_url = ngrok.connect(addr='8501')
print(f'Access the app here: {public_url}')

Access the app here: NgrokTunnel: "https://b14b-34-122-214-116.ngrok-free.app" -> "http://localhost:8501"
